# Reverse2 - Volatility

### 1er étape :  Nom de la machine

Nous allons dans un premier temps executer une commande pour avoir les informations générales de la machine.

Pas beaucoup d'information sur notre nom de machine avec cette commande. On sait que le nom de la machine est contenue dans le registre et plus précisemment dans \REGISTRY\MACHINE\SYSTEM\

In [ ]:
import os
def windows_info():
    os.system("vol -f dump.vmem windows.info > window-info-output.txt")
    output = os.popen("vol -f dump.vmem windows.info").read()
    print(output)

windows_info()


vol -f dump.vmem windows.registry.printkey. --offset 0xf8a000024010 --key "ControlSet001\Control\ComputerName\ComputerName"
Cette commande nous permet d'afficher le contenu de la variable **ComputerName** contenue dans le registre MACHINE\SYSTEM
Nom de la machine : WIN-LO6FAF3DTFE

In [ ]:
def windows_getkey():
    os.system("vol -f dump.vmem windows.registry.printkey.PrintKey > window-getkey-output.txt")
    os.system('vol -f dump.vmem windows.registry.printkey. --offset 0xf8a000024010 --key "ControlSet001\Control\ComputerName\ComputerName" > window-getmachinename-output.txt')
    computername = os.popen('vol -f dump.vmem windows.registry.printkey. --offset 0xf8a000024010 --key "ControlSet001\Control\ComputerName\ComputerName"').read()
    print(computername)

windows_getkey()

### 2e étape :  Credentials

Pour ce faire on utilise hashdump qui nous récupère les hash des mots de passes de chaques sessions stocké.

In [ ]:
def getUserCredentials():
    os.system("vol -f dump.vmem windows.hashdump.Hashdump  > hashes.txt")
    output = os.popen("vol -f dump.vmem windows.hashdump.Hashdump").read()
    credentials = os.popen("vol -f dump.vmem windows.lsadump").read()
    print(credentials)

getUserCredentials()